## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.6+ MB


In [4]:
data

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Ознакомившись с полученными данными, выявлены следующие проблемы: 
1) Некорректные данные в столбце 'days_employed', которые скорее всего не несут в себе никакой полезной
для нас информации, кроме той, что скорее всего это вызвано ошибкой при импорте / экспорте данных. 

2) Столбец 'education' имеет запись в разном регистре, это нужно исправить для удобной работы с данными
на следующих этапах. 

3) 'total_income' имеет вещественный тип данных, с ним можно работать, но для удобства лучше перевести 
столбец к целочисленному типу данных. Каких-либо последствий это не вызовет, а комфортности для анализа добавит. 

4) 'purpose' имеет много разных записей относящихся, по факту, к нескольким категориям. Эти данные мы будем 
категоризировать. Первичная ситуация с данными ясна и пока всё довольно ясно - нужно привести данные в порядок,
привести строки к определенному формату записи для каждого типа данных, выявить дубликаты и пропуски.

<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>
<font color='green'>
ОК, это есть, ну и плюс ещё всякого.<br>
На этом этапе лучше прописывать всё что увидели по каждой переменной. В итоге будет как план к действию)))
<br>
</font>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
data['children'] = data['children'].replace(-1 , 0)
data['children'] = data['children'].replace(20, 2)

In [6]:
data['education'] = data['education'].str.lower()
data['purpose'] = data['purpose'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()

In [7]:
data[data['dob_years'] == 0].count()
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()
data['dob_years'] = round(data['dob_years'])
data['dob_years'] = data['dob_years'].astype('int64')

In [8]:
data['gender'].isna().sum()
data['gender'].unique()
data['gender'] = data['gender'].replace('XNA', 'F')

### Обработка дубликатов

In [9]:
data.duplicated().sum()
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21454 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.7+ MB


### Замена типа данных

In [11]:
income_type_group = data.groupby('income_type')['total_income'].mean()
print('Пропусков до обработки "total_income":', data['total_income'].isna().sum())

for income_type in income_type_group.index:
    data.loc[data["income_type"] == income_type, "total_income"] = data.loc[data["income_type"] == income_type, "total_income"].fillna(income_type_group.loc[income_type])
    
print('Пропусков после обработки "total_income":', data['total_income'].isna().sum())

Пропусков до обработки "total_income": 2103
Пропусков после обработки "total_income": 0


In [ ]:
print('Пропусков до обработки "days_employed":', data['days_employed'].isna().sum())
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].mean())
print('Пропусков после обработки "days_employed":', data['days_employed'].isna().sum())

In [ ]:
data['total_income'] = data['total_income'].astype('int64')
data['days_employed'] = data['days_employed'].astype('int64')

In [ ]:
data.info()

In [ ]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

### Лемматизация

In [ ]:
text = ' '.join(data['purpose'])
lemmas = m.lemmatize(text)
print(Counter(lemmas))

### Категоризация данных

In [31]:
def lemma_func(lemmas):
    for word in lemmas.split(' '): 
        if 'движ' in word or 'жи' in word:
            return 'недвижимость'
        if 'авто' in word:
            return 'автомобиль'
        if 'свадь' in word:
            return 'свадьба'      
    return 'образование'


def income_func(income):
    if income < 48000:
        return 'ниже среднего'
    return 'выше среднего'


data['purpose_reason'] = data['purpose'].apply(lemma_func)
data['purpose'] = data['purpose_reason']
data.drop(['purpose_reason'], axis='columns', inplace=True)

data['income_level'] = data['total_income'].apply(income_func)

In [32]:
data.duplicated().sum()
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21116 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21116 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21116 non-null  int64  
 3   education         21116 non-null  object 
 4   education_id      21116 non-null  int64  
 5   family_status     21116 non-null  object 
 6   family_status_id  21116 non-null  int64  
 7   gender            21116 non-null  object 
 8   income_type       21116 non-null  object 
 9   debt              21116 non-null  int64  
 10  total_income      21116 non-null  float64
 11  purpose           21116 non-null  object 
 12  income_level      21116 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.8+ MB


Финальный шаг перед расчётами. Написаны две функции. Первая функция - это перебор однокоренных слов с помощью цикла 
и метода '.split()' в списке 'purpose' для создания столбца с категоризироваными данными. 
Вторая функция - это категоризация уровня дохода для общей информации клиентской базы

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
children_debt_pivot = data.pivot_table(index='children', columns='debt', values='dob_years', aggfunc='count')
children_debt_pivot['ratio'] = 1 - children_debt_pivot[1] / children_debt_pivot[0]
children_debt_pivot = children_debt_pivot.fillna(0)
children_debt_pivot.sort_values(by='ratio', ascending = False)

debt,0,1,ratio
children,,,
0,13074.0,1064.0,0.918617
3,303.0,27.0,0.910891
1,4364.0,444.0,0.898258
2,1926.0,202.0,0.895119
4,37.0,4.0,0.891892
5,9.0,0.0,0.000000


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [28]:
family_status_debt_pivot = data.pivot_table(index='family_status', columns='debt', values='dob_years', aggfunc='count')
family_status_debt_pivot['ratio'] = 1 - family_status_debt_pivot[1] / family_status_debt_pivot[0]
family_status_debt_pivot = family_status_debt_pivot.fillna(0)
family_status_debt_pivot.sort_values(by='ratio', ascending = False)

debt,0,1,ratio
family_status,,,
вдовец / вдова,896,63,0.929688
в разводе,1110,85,0.923423
женат / замужем,11408,931,0.918391
гражданский брак,3763,388,0.896891
не женат / не замужем,2536,274,0.891956


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [35]:
income_debt_pivot = data.pivot_table(index='income_level', columns='debt', values='dob_years', aggfunc='count')
income_debt_pivot['ratio'] = 1 - income_debt_pivot[1] / income_debt_pivot[0]
income_debt_pivot = income_debt_pivot.fillna(0)
income_debt_pivot.sort_values(by='ratio', ascending = False)

debt,0,1,ratio
income_level,,,
ниже среднего,268,19,0.929104
выше среднего,19109,1720,0.909990


### Вывод

По процентному соотношению прямой зависимости между возвратом кредита в срок и уровнем дохода не выявлено

- Как разные цели кредита влияют на его возврат в срок?

In [36]:
purpose_debt_pivot = data.pivot_table(index='purpose', columns='debt', values='dob_years', aggfunc='count')
purpose_debt_pivot['ratio'] = 1 - purpose_debt_pivot[1] / purpose_debt_pivot[0]
purpose_debt_pivot = purpose_debt_pivot.fillna(0)
purpose_debt_pivot.sort_values(by='ratio', ascending = False)

debt,0,1,ratio
purpose,,,
недвижимость,9794,781,0.920257
свадьба,2120,186,0.912264
образование,3594,370,0.897051
автомобиль,3869,402,0.896097
